### Setup

In [1]:
%xmode plain
from google.colab import drive
drive.mount('/content/drive')
!pip install sktime

Exception reporting mode: Plain
Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████████████████████████████| 9.8 MB 39.3 MB/s 
     |████████████████████████████████| 34.5 MB 10 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


### Read in data

In [23]:
import os
import re
import pandas as pd
import numpy as np
from glob import glob

feature_df = pd.read_csv(
    "/content/drive/MyDrive/IOT Classification Challenge/feature_df_951.csv", 
    header = 0,
    index_col = 0
)
temp_label_df = pd.read_csv(
    "/content/drive/MyDrive/IOT Classification Challenge/label_df_951.csv", 
    header = 0, 
    index_col = 0
)
label_df = pd.Series(data = temp_label_df['0'], index = temp_label_df.index)

### Reconstruct the multivariate time series

In [26]:
def reshape_mtsc(feature_df, label_df):
    mts_stack = []
    for video in feature_df.index:
        feature_case = np.array(feature_df.loc[video,:]).reshape((34,100)) # reshape to (34,100)
        mts_stack.append(feature_case)
        label_case = label_df[video]
    return np.stack(mts_stack, axis = 0)

mts_stack = reshape_mtsc(feature_df, label_df) # (951, 34, 100)

### Train test split

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from matplotlib import cm, pyplot as plt

mts_train, mts_test, label_train, label_test = train_test_split(
    mts_stack, label_df, 
    test_size = 0.2, 
    random_state = 42,
    shuffle = True
)

### Random Convolutional Kernel Transform

In [30]:
from tqdm import tqdm
from time import sleep
import numpy as np
import math
import sys
from numba import njit
from sktime.classification.kernel_based import RocketClassifier # import need 2 min

rocket_clf = RocketClassifier(
    num_kernels = 10000, 
    random_state = 42, 
    n_jobs = -1
)

mts_train_np = np.asarray(mts_train)
mts_test_np = np.asarray(mts_test)
label_train_np = np.asarray(label_train)

rocket_clf.fit(mts_train_np, label_train_np)
label_pred = rocket_clf.predict(mts_test_np)

In [31]:
accuracy_score(y_pred = label_pred, y_true = label_test)

0.7853403141361257